In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,sys
import argparse
import cv2
import pickle
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [17]:
class Shape_classifier(nn.Module):
    def __init__(self):
        super(Shape_classifier, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=2)
        self.conv31 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=2)
        self.pool3 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.conv4 = nn.Conv2d(128, 196, kernel_size=3, stride=1, padding=2)
        self.pool4 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.fc1 = nn.Linear(7056, 2048)
        self.fc2 = nn.Linear(2048, 9)

        self.drop = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv31(x))
        x = self.pool3(x)

        x = F.relu(self.conv4(x))
        x = self.pool4(x)

        x = x.reshape(x.size(0), -1)
        x = self.drop(x)
        x = F.relu(self.fc1(x))

        x = self.fc2(x)
        return x

In [3]:
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize(100),
                                transforms.ToTensor(),
                                transforms.Normalize((0,), (1,))])

In [21]:
net = Shape_classifier()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Shape_classifier(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (pool1): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv31): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (pool3): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 196, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (pool4): MaxPool2d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=7056, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=9, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
)


In [73]:
# Specify a path
PATH = "./model.pth"

model = torch.load(PATH)

In [82]:
curr_dir = os.getcwd()
rel_path = os.path.join(curr_dir, "testing")
image_filenames = []
def extract_edge(images_path):
    data = []
    for element in os.listdir(images_path):
        image_filenames.append(element)
        path = os.path.join(images_path,element)
        m = np.ones((2, 2),np.uint8)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        edges = cv2.Canny(img,100,200)
        edges = cv2.dilate(edges,m,iterations=2)
        data.append((element.split("_")[0].lower(),edges))
    return data,image_filenames

edges_data,image_filenames = extract_edge(rel_path)
with open("data_test.pickle", "wb") as h:
    pickle.dump(edges_data, h, protocol=pickle.HIGHEST_PROTOCOL)

In [78]:
class geometryShapes:

    def __init__(self, transform):
        self.transform = transform

        with open("data_test.pickle", "rb") as h:
            self.constituents = pickle.load(h)
            
        
        self.classes = {'pentagon': 0,
                        'circle': 1,
                        'nonagon': 2,
                        'triangle': 3,
                        'octagon': 4,
                        'square': 5,
                        'heptagon': 6,
                        'hexagon': 7,
                        'star': 8}

    def __len__(self):
        return len(self.constituents)

    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        i = Image.fromarray(self.constituents[idx][1], 'L')
        tensor_img = self.transform(i)
        torch.set_printoptions(profile="full")
        sample = {"label": self.classes[self.constituents[idx][0]], "picture": tensor_img}
        return sample

    def split_training_testing(self):
        subsets = torch.utils.data.random_split(self, [int(len(self))])
        datasets = self
        return datasets

In [79]:
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize(100),
                                transforms.ToTensor(),
                                transforms.Normalize((0,), (1,))])

In [80]:
ds = geometryShapes(transform)
datasets = ds.split_training_testing()
dataloaders = torch.utils.data.DataLoader(datasets, batch_size=1, shuffle=True)
l = len(datasets)
print(l)

4


In [91]:
classes = {0 : 'pentagon',
            1: 'circle' ,
            2:'nonagon',
            3:'triangle',
            4:'octagon',
            5:'square',
            6:'heptagon',
            7:'hexagon',
            8:'star'}

In [96]:
with torch.no_grad():
    model.cpu()
    for i in range(l):
        item = datasets[i]
        image = item['picture'].unsqueeze_(0)
        prediction = model(image)
        predicted_class = np.argmax(prediction).detach().numpy()
        print("Filename : ",image_filenames[i]," prediction: ",classes[int(predicted_class)])

Filename :  Circle_003fd9dc-2a99-11ea-8123-8363a7ec19e6.png  prediction:  circle
Filename :  Pentagon_47b6cfa6-2a84-11ea-8123-8363a7ec19e6.png  prediction:  pentagon
Filename :  Square_e4dad8a2-2a95-11ea-8123-8363a7ec19e6.png  prediction:  square
Filename :  Triangle_fffd8096-2a89-11ea-8123-8363a7ec19e6.png  prediction:  triangle
